In [3]:
# v1
# update file
import re
import json
from google import genai
from google.genai import types

system_prompt = """Translate texts from English to Vietnamese.
The subject matter is Early Buddhism. Translations should be *easy to understand, using simple, modern, everyday language*.

RULE:
1. Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

2. Maintain the original paragraph structure; do not add or remove any text.

3. Key Term Explanation, explaining terms that are central to Early Buddhist thought and likely unfamiliar to a 10th-grader. won't explain every single
Pali word, only the ones that are crucial for understanding.
When translation, if key terms or concepts that require clarification then add explanations, add explanations will be short and to the point, avoiding philosophical debates.
Example, If the text says "the cessation of craving," I might translate it as: "sự chấm dứt tham ái (craving - sự ham muốn mạnh mẽ)."
But if it says "suffering," I likely won't add an explanation unless the context makes it unclear, because "khổ" is a common word.

4. Glossary (Strict Format): In the end, add The glossary, glossary will only include terms that were actually used and explained within the translated text itself.
**Format:** `Vietnamese Term / Pali Term / English Term: Expanded explanation from the in-text definition.`
No generated content: The explanation in Vietnamese must be consistent to the meaning in the text, no unrelated content.

```

<!--pg-->
Từ ngữ:

- con đường tám nhánh / aṭṭhaṅgika magga / eightfold path: Tám phương diện tu tập để đạt đến giác ngộ.
- giác ngộ / bodhi / enlightenment: Trạng thái hiểu biết hoàn toàn về sự thật.
- thiền định / samādhi / concentration: Tập trung tâm trí vào một đối tượng duy nhất.
"""

system_prompt2 = """Translate the following English text about Early Buddhism into Vietnamese.
The translation should be suitable for a 10th-grade reading level, using simple, modern, and everyday language.

RULE:

- Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

- **Paragraph Structure:** Maintain the original paragraph structure. Do not add or remove any text.

- **Readability:** Use clear, simple Vietnamese. Avoid complex or archaic vocabulary.

- **Key Term Explanations (IN-TEXT):**  Identify terms *crucial* to understanding Early Buddhist thought that a 10th-grader likely wouldn't know. Explain these terms *within the translation itself*, immediately after their first use. Keep explanations short and to the point.
    *   **Format:** `Vietnamese Term (Pali Term - English Term: short definition)`
    *   **Example:** Instead of just "cessation of craving," translate it as:  `sự chấm dứt tham ái (taṇhā - craving: sự ham muốn mãnh liệt, dai dẳng).`
    *   **Do NOT explain very common words** like "suffering" (`khổ`) unless the context is unusual. Only explain terms related to early Buddhism context.

- **Glossary (Strict Format):**
    *   **AFTER** the translation, create a glossary.
    *   **ONLY include terms explained IN-TEXT (Rule 3).**
    *   **Format:** `Vietnamese Term / Pali Term / English Term: Expanded explanation from the in-text definition.`

**Glossary Example (Following a Hypothetical Translation):**

<!--pg-->
Từ ngữ:

- sự chấm dứt tham ái / taṇhā / craving: Sự ham muốn mãnh liệt, dai dẳng, nguyên nhân chính của khổ đau.
- bốn sự thật cao quý / cattāri ariya saccāni / four noble truths: Bốn sự thật mà Đức Phật đã giác ngộ, nền tảng của Phật giáo.
- con đường tám nhánh / aṭṭhaṅgika magga / eightfold path: Tám phương diện tu tập để đạt đến giác ngộ.
- giác ngộ / bodhi / enlightenment: Trạng thái hiểu biết hoàn toàn về sự thật.
- thiền định / samādhi / concentration: Tập trung tâm trí vào một đối tượng duy nhất.
"""

system_prompt3 = """Translate content from English to Vietnamese, focusing on Theravada Buddhist texts. Use simple,
understandable, modern Vietnamese terminology for accessibility. For specialized Buddhist terms that are uncommon in
everyday language (such as "attachment" or "complete nirvana" or "purification of doubt"), please provide additional
explanation by including direct English and raw-explaination in vietnam translation in parentheses but only for the first occurrence of each term (explain once).

Example format 1: "tỳ kheo (monks-bhikkhus-nhà sư)" or "tỳ kheo (bhikkhus)",
Example format 2: "thanh tịnh bằng tri kiến về đạo lộ (purification by knowledge and vision of the way - sự thanh lọc bằng kiến ​​thức và tầm nhìn về đường đạo)"

At the end of each translation, please include a glossary section with the following format:

<!--pg-->
Từ ngữ:
- Vietnamese term / Pali term / English term: Comprehensive explanation in Vietnamese
Example:
- bốn sự thật cao quý / cattāri ariya saccāni / four noble truths: Bốn sự thật mà Đức Phật đã giác ngộ, nền tảng của Phật giáo.

- Always maintain the original meaning while making the translation accessible to modern Vietnamese readers.
When encountering ambiguous terms, provide multiple possible interpretations if necessary.
For quoted passages, preserve the poetic quality where possible while ensuring clarity.
Focus particularly on accuracy with technical dharma terms and use simple, understandable, modern Vietnamese terminology for accessibility.
Maintain the original paragraph structure.
"""
# Focus particularly on accuracy with technical dharma terms and use simple,
# understandable, modern Vietnamese terminology for accessibility

system_prompt_note = """Translate texts from English to Vietnamese.
The subject matter is Early Buddhism. Translations should be *easy to understand, using simple, modern, everyday language*.

RULE:
1. Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

2. Maintain the original paragraph structure; do not add or remove any text.

3. When translation, if key terms or concepts that require clarification, please provide the Vietnamese translation,
followed by the original English word in (), and a brief explanation if necessary.
Example1: `giới cấm thủ (adherence to rules and observances-sự tuân thủ các quy tắc và việc thực hiện các nghi lễ hoặc phong tục)`
Example2: `sân hận (ill will)`
"""

response = None
client = None
def translate_markdown(input_file: str, output_file: str, api_key: str, prompt, model, log, temperature=0):
    """
    Translates an English markdown file to Vietnamese using Google GenAI.

    Args:
        input_file: Path to the input English markdown file (e.g., "abc.md").
        output_file: Path to the output Vietnamese markdown file (e.g., "abc.vn.md").
        api_key: Your Google GenAI API key.
    """

    global response
    global client

    client = genai.Client(api_key=api_key)

    file = client.files.upload(file=input_file)
    # --- Core Translation Logic ---
    try:
        err = ""
        response = client.models.generate_content(
            model= model,
            contents=[file],
            config=types.GenerateContentConfig(
                system_instruction=prompt,
                max_output_tokens=50000,
                # candidates_token_count = 20000,
                # input_token_limit = 20000,
                temperature=temperature
                # temperature=1
                ),
        )

        if response.candidates:
            if response.candidates[0].finish_reason != types.FinishReason.STOP:
                print(f": {input_file} resonse: {response.candidates[0].finish_reason}")
                # log.write(f"{filename}, exception: File not found\n")
                log.write(f": {input_file} resonse {response.candidates[0].finish_reason}")
                #If it is not stop, consider reducing the chunk size
        else: #No candidated return
            pass
            # err = "response.candidates is empty. reduce the chunk size"


        vietnamese_markdown = response.text

    except Exception as e:
      log.write(f'f"Error during translation: {e}')
      print(f'f"Error during translation: {e}')
    finally:
         # --- Output Handling ---
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(vietnamese_markdown)
                log.write(f'{input_file} ok')
        except Exception as e:
            log.write(f'{input_file} Error writing output file: {e}')


    # return f'{input_file} {output_file}:{err}'


import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# peterontech key

api_key = os.environ.get("GOOGLE_API_KEY")
temperature = 0.05
model= "gemini-2.0-pro-exp-02-05"
# model ="gemini-2.0-flash-thinking-exp-01-21"
# model = "gemini-2.0-flash"
# model="gemini-2.0-flash"

def trans_file():

    fname = "033-the-greater-discourse-on-the-cowherd"

    input_filename  = f"../docs/kinhtrungbo/nanamoli-bodhi-en/{fname}.md"
    output_filename = f"../docs/kinhtrungbo/nanamoli-bodhi-vi/{fname}.md"
    # fname = "001"
    # input_filename  = f"../kinhtrungbo/nanamoli-bodhi-en/outro/notes/{fname}.md"
    # output_filename = f"../kinhtrungbo/nanamoli-bodhi-vi/outro/notes/{fname}.md"


    fi = open("tran_ressult.txt", "w")
    prompt = system_prompt3
    # prompt = system_prompt_note
    translate_markdown(input_filename, output_filename, api_key, prompt, model, fi, temperature)
    fi.close()

trans_file()


In [10]:

import os
def trans_folder(infilelist, output_folder, limit=None, test=False):
    fi = open("tran_ressult.txt", "w")

    count = 1
    for filepath in infilelist: # os.listdir(input_folder):
        filename = os.path.basename(filepath)

        if limit is not None and count >= limit:
            break  # Stop if the limit is reached

        # input_filename = os.path.join(input_folder, filename)
        output_filename = os.path.join(output_folder, filename)

        print(f"Translating {filepath} to {output_filename}")

        try:
            # pass
            if test is False:
                translate_markdown(filepath, output_filename, api_key, system_prompt, model, fi)
            else:
                print(filepath)
            count += 1
        except Exception as e:
            print(f"Error processing {filename}: {e}")  # Handle potential errors
    fi.close()


in_folder  = '../kinhtrungbo/nanamoli-bodhi-en/outro/'
out_folder = '../kinhtrungbo/nanamoli-bodhi-vi/outro/'
filelist = []

for filename in sorted([f for f in os.listdir(in_folder) if os.path.isfile(os.path.join(in_folder, f))]):
    filelist.append(f'{in_folder}{filename}')
# filelist
# filelist = filelist[1:]
trans_folder(infilelist=filelist, output_folder=out_folder)


Translating ../kinhtrungbo/nanamoli-bodhi-en/outro/1.bibliography.md to ../kinhtrungbo/nanamoli-bodhi-vi/outro/1.bibliography.md
Translating ../kinhtrungbo/nanamoli-bodhi-en/outro/2.list-of-abbreviations.md to ../kinhtrungbo/nanamoli-bodhi-vi/outro/2.list-of-abbreviations.md
Translating ../kinhtrungbo/nanamoli-bodhi-en/outro/3.notes.md to ../kinhtrungbo/nanamoli-bodhi-vi/outro/3.notes.md
Translating ../kinhtrungbo/nanamoli-bodhi-en/outro/4.pali-english-glossary.md to ../kinhtrungbo/nanamoli-bodhi-vi/outro/4.pali-english-glossary.md


In [15]:
# get all file
filelist = []
for filename in sorted(os.listdir('../kinhtrungbo/nanamoli-bodhi-en')):
    filelist.append(filename)

for filename in sorted([f for f in os.listdir('../kinhtrungbo/nanamoli-bodhi-en') if os.path.isfile(os.path.join('../kinhtrungbo/nanamoli-bodhi-en', f))]):
    filelist.append(filename)
